In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from fastapi import FastAPI, File, UploadFile
import uvicorn
import json
from fastapi.responses import PlainTextResponse
import matplotlib.pyplot as plt
import numpy as np

In [2]:
path = "server"
models = "models"

In [3]:
import os
from os import listdir

#создание папок для набора данных и моделей
if not os.path.exists(path):
    os.makedirs(path)
if not os.path.exists(models):
    os.makedirs(models)

In [4]:
#сохранение csv файла без сжатия
def uploadFile(df, filename):
    df.to_csv(f"{path}/{filename}", index=False)
    return filename

In [5]:
#сохранение csv файла с сжатием в pkl
def uploadFileP(df, filename):
    filename = filename.replace(".csv", ".pkl")
    df.to_pickle(f"{path}/{filename}")
    return f'Файл обработан, размер данных на сервере {os.path.getsize(f"{path}/{filename}")} байт'

In [ ]:
app = FastAPI()

model = -1
data = -1
ld = -1
res = -1
ch = -1

#установка текущего набора данных с проверкой наличия файла с таким именем
def setF(filename):
    global res
    global data
    global ld
    global ch
    ch = -1
    if not os.path.isfile(f"{path}/{filename}"):
        return "Файл не найден"
    if filename.find(".csv") != -1:
        data = pd.read_csv(f"{path}/{filename}")
        ld = 1
        return "Данные загружены в память"
    if filename.find(".pkl") != -1:
        data = pd.read_pickle(f"{path}/{filename}")
        ld = 1
        return "Данные загружены в память"
    return "Формат файла не поддерживается"

#установка текущей модели с проверкой наличия файла с таким именем
def setM(filename):
    global model
    global ch
    ch = -1
    if not os.path.isfile(f"{models}/{filename}"):
        return "Файл не найден"
    model = CatBoostClassifier()
    model.load_model(f"{models}/{filename}")
    return filename

# предсказание для текущего набора данных по текущей модели
def pr():
    global ch
    if model == -1:
        return "Не выбрана модель"
    if ld == -1:
        return "Не выбран набор данных"
    ch = 1
    res = model.predict(data)
    return json.dumps(res.reshape(-1).tolist())

# постройка точечной диаграммы по параметру и предсказанию, диаграмма переводится в numpy массив и парсится в json
def plot(name):
    global data
    global model
    global ch
    if ch == -1:
        return "Набор данных или модель были изменены / не установлены"
    if not name in data.columns:
        return "В наборе данных параметра с таким названием нет"
    plt.scatter(data[name], model.predict(data))
    fig = plt.figure()
    fig.canvas.draw()

    d = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    d = d.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    return json.dumps(d.tolist())

def ft(name):
    global data
    global model
    global ch
    if ch == -1:
        return "Набор данных или модель были изменены / не установлены"
    if not name in data.columns:
        return "В наборе данных параметра с таким названием нет"
    return json.dumps(data[name].values.tolist())
    
    
# вывод информации о методах Api
@app.get("/methodsInfo", response_class=PlainTextResponse)
def methodsInfo():
    res = "Методы API:\n"
    res += "0. methodsInfo - выводит информацию обо всех методах системы\n"
    res += "1. info - возвращает информацию о версии системы, ее авторе и организации разработчике\n"
    res += "2. upload - загружает файл с набором данных в формате csv с сохранением без сжатия, возвращает название файла\n"
    res += "3. uploadP - загружает файл с набором данных в формате csv с сохранением в формате pkl, возвращает размер сжатого файла\n"
    res += "4. allFiles - возвращает список загруженных на сервер наборов данных\n"
    res += "5. setFile - принимает название файла, загружает данные из него в текущие данные, если файла нет - выводит ошибку\n"
    res += "6. allModels - возвращает список загруженных на сервер моделей\n"
    res += "7. setModel - принимает название модели, загружает ее в текущую модель, если файла нет - выводит ошибку\n"
    res += "8. predict - выполняет прогнозирование на текущих данных при помощи текущей модели и возвращает массив с результатами прогноза, если модель / данные не установлены выдает информацию об этом\n"
    res += "9. img - получает название параметра и на его основе возвращает график зависимости результата прогноза от этого параметра, если параметра в данных нет или нет прогноза модели для текущих данных возвращает ошибку\n"
    return res

# вывод информации о версии и разработчике
@app.get("/info")
def info():
    return "Версия - 1.0.0. Автор - Сизяков Иван Романович, 2-42м, организация-разработчик - ИГЭУ"

# загрузка файла данных без сжатия
@app.post("/upload")
def upload(file: UploadFile = File(...)):
    df = pd.read_csv(file.file)
    return uploadFile(df, file.filename)

# загрузка файла данных для дальнейшего сжатия
@app.post("/uploadP")
def uploadP(file: UploadFile = File(...)):
    df = pd.read_csv(file.file)
    return uploadFileP(df, file.filename)

# получение списка всех наборов данных
@app.get("/allFiles")
def allFiles():
    return listdir(path)

# установка текущего набора данных
@app.get("/setFile")
def setFile(filename):
    return setF(filename)

# вывод списка всех моделей на сервере
@app.get("/allModels")
def allModels():
    return listdir(models)

# установка текущей модели
@app.get("/setModel")
def setModel(filename):
    return setM(filename)

# выполнение прогнозирования
@app.get("/predict")
def predict():
    return pr()

# постройка графика по прогнозу и параметру
@app.get("/img")
def img(name):
    return plot(name)

@app.get("/feature")
def feature(name):
    return ft(name)

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [2448]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50092 - "GET /allModels HTTP/1.1" 200 OK
INFO:     127.0.0.1:50095 - "GET /allFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:50096 - "GET /allModels HTTP/1.1" 200 OK
INFO:     127.0.0.1:50097 - "GET /allFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:50099 - "GET /allFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:50102 - "GET /allModels HTTP/1.1" 200 OK
INFO:     127.0.0.1:50103 - "GET /allFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:50117 - "GET /setFile?filename=test_shapes.pkl HTTP/1.1" 200 OK
INFO:     127.0.0.1:50121 - "GET /allFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:50125 - "GET /allModels HTTP/1.1" 200 OK
INFO:     127.0.0.1:50126 - "GET /setModel?filename=meow1000.bin HTTP/1.1" 200 OK
INFO:     127.0.0.1:50127 - "GET /setFile?filename=train_data.csv HTTP/1.1" 200 OK
INFO:     127.0.0.1:50128 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0.0.1:50140 - "GET /feature?name=feature_1 HTTP/1.1" 200 OK
INFO:     127.0.0.1:50141 - "GET /predict HTTP/1.1" 200 OK
INFO:     127.0